In [56]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import warnings

warnings.filterwarnings("ignore")

ind_bla, pac_bla, org_bla, bla_cands = pre_process_az()

This notebook examines the overall spending of PACs and organizations, the contributions of individuals, and the income of candidates within the 2014-22 period. To reproduce the results, go to the bottom and load the datasets there. 

### Is the Dataset Relational?

The Arizona campaign finance dataset is not relational. While it includes many different tables, each of these is an aggregate of bulk transaction data (presently unavailable). The detailed transaction tables themselves are only subtables, and do not have a relational structure. 

### Columns and NaNs

Of the four aggregate tables examined in the period 2014-2022:

**Candidates:** Name (str), Candidate Committee (str), Office (str, 20 missing values), Party (str, 62 missing values), Income (float), Expense (float), IEFor (float), IEAgainst (float)

**Organizations:** Name(str), Cash Balance (float), IEFor (float), IEAgainst (float), BMEFor (float), BMEAgainst (float) (no missing values)

**PACs:** Name(str), Income (float), Expense (float), Cash Balance (float), IEFor (float), IEAgainst (float), BMEFor (float), BMEAgainst (float) (no missing values)

**Individuals:** Name(str), Amount (float, no missing values)



### Top Individual Contributors 2014-22


Top Individual Contributors in Arizona from 2014-22:

1. Karron Taylor Robson, $18.90 million

2. Christine N Jones, $4.82 million

3. Karla T Jurvetson, $4.12 million

4. Steve Gaynor, $3.74 million

5. Katherin Leslie Rudin, $3.49 million

6. Stacy Schustermann, $3.40 million

7. Michael Bloomberg, $3.04 million

8. Robert Parsons, $2.53 million

9. Lynn Schustermann, $2.53 million

10. Barry Munger, $2.50 million

In [37]:
# looking at just proper contributors, ignoring 'multiple contributors' in first place
ind_bla[1:11]

,total_spending,type
Name,,
"robson, karrin taylor",18893866.96,Individual
"jones, christine n.",4815696.86,Individual
"jurvetson, karla t.",4116410.28,Individual
"gaynor, steve",3743548.74,Individual
"rudin, katherine leslie",3490000.00,Individual
"schusterman, stacy",3400900.00,Individual
"bloomberg, michael",3035000.00,Individual
"parsons, robert",2958130.00,Individual
"schusterman, lynn",2531500.00,Individual


### Top Overall Spenders 2014-22

Top Spenders across Individual Donors, PACs and Organizations
Note that PACs and Organizations are not listed as making 'contributions', only expenditures and other expenses

1. Arizonans for Affordable Electricity (PAC), $64 million

2. Republican Governors Association (RGA) PAC (PAC), $46 million

3. Invest in Arizona (Sponsored by AEA and Stand for Children) (PAC), $41 million

4. The PAC for America's Future - AZ (PAC), $31 million

5. Clean Energy for a Healthy Arizona (PAC), $28 million

6. Worker Power PAC, $21 million

7. Arizonans Fed Up with Failing Healthcare (Healthcare Rising AZ) (PAC), $20 million

8. Karrin Taylor Robson (Individual), $18 million

9. Citizens forFair Tax Policy (PAC), $18 million 

10. Chispa AZ PAC (PAC), $16 million

Only 1 individual cracks the top ten, and no 'organizations' do the same. 

In [55]:
# looking at all expenses by individual contributors, PACs and organizations
top_expenses = (
    ind_bla.append(org_bla)
    .append(pac_bla)
    .sort_values(by="total_spending", ascending=False)
)

top_expenses.head(11)

,total_spending,type
Name,,
arizonans for affordable electricity,64046487.54,PAC
rga arizona pac,46091800.28,PAC
invest in arizona (sponsored by aea and stand for children),41360742.64,PAC
multiple contributors,38707746.13,Individual
the pac for america's future - az,31260121.77,PAC
clean energy for a healthy arizona,28924859.15,PAC
worker power pac,21843510.03,PAC
arizonans fed up with failing healthcare (healthcare rising az),20023103.55,PAC
"robson, karrin taylor",18893866.96,Individual


### Recipients by Office

In [36]:
# means

grp_cands = cands_14_22.groupby(by="Office Summary").mean()

fig = px.bar(
    x=list(grp_cands.index),
    y=grp_cands["Expense"].values,
    title="Mean Candidate Income by Office, 2014-22",
    labels={"x": "Year", "y": "US Dollars"},
)

fig.show()

Above we see that, in terms of mean income by office type across the studied period, candidates for governor received the most, at over 600k on avergae, followed shortly by candidates for Attorney General at a little under 500k, and Secretary of State around 350k. Other offices receive much less on average, with none breaking 100k mean, and the average State Senator and State Representative receiving less than 50k across the period. 

In [35]:
grp_cands = cands_14_22.groupby(by="Office Summary").sum()

fig = px.bar(
    x=list(grp_cands.index),
    y=grp_cands["Expense"].values,
    title="Gross Candidate Income by Office, 2014-22",
    labels={"x": "Year", "y": "US Dollars"},
)

fig.show()

Above we see that, in terms of total income by office type across the studied period, candidates for Governor took in over 90 million dollars, a near majority of all candidate income. State Representatives and State Senators took around 35 and 30 million in the same period, as there are a great many such offices.

### Spending over the Years

As seen below, spending and income is not static over the 2014-22 period, with the most recent election cycle in 2022 receiving a massively outsized amount of spending. This is partly explicable by election schedules, but not entirely; it appears that Arizona has rapidly become a much more contested state with much more campaign expenditure within the last several years. 

In [34]:
# recipient income by year

by_year_cands_sum = cands_14_22.groupby(by="Year").sum()

fig = px.bar(
    x=list(by_year_cands_sum.index),
    y=by_year_cands_sum["Income"].values,
    title="Gross Candidate Income by Year, 2014-22",
    labels={"x": "Year", "y": "US Dollars"},
)

fig.show()

Above we see that total candidate income is very jagged across the studied period. Even-numbered election years reliably take in at least twice as much spending as the preceding odd-numbered year, but an overall trend is unclear. For example, 2014 income, at 35 million dollars, was not exceeded until the outlier year 2022. That said, an overall increase within the latter half of the period does seem apparent, with overall income at least doubling every year from 2019 to 2022.

In [29]:
# individual contributions by year

grp = inds_14_22.groupby(by="Year").sum()

grp["total_spending"].values

fig = go.Figure(
    data=[
        go.Bar(
            name="Individual",
            x=list(range(2014, 2023)),
            y=grp["total_spending"].values,
        )
    ]
)

fig.update_layout(
    title="Gross Individual Contributions by Year, 2014-22",
    xaxis_title="Year",
    yaxis_title="US Dollars",
    #     legend_title="Legend Title",
    #     font=dict(
    #         family="Courier New, monospace",
    #         size=18,
    #         color="RebeccaPurple"
    #     )
)

fig.show()

Similar to the graph of candidate income year to year, we see considerable jaggedness within the studied period, with a huge peak in 2022. Some notable differences include a derease in individual contributions from 2020 to 2021,and 2018 contributions exceeding 2014. It is also worth noting that individual contributions exceed candidate income each year by around 50%, if not more, as these contributions also go to other recipients such as PACs. 

In [43]:
# overall expenses by type by year

donors_14_22 = (
    org_14_22[["Name", "total_spending", "type", "Year"]]
    .append(pac_14_22[["Name", "total_spending", "type", "Year"]])
    .append(inds_14_22[["Name", "total_spending", "type", "Year"]])
)

donors_14_22.groupby(by=["type", "Year"]).sum()

years = ["2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022"]

yrs = []

df = donors_14_22.groupby(by=["type", "Year"]).sum()

for i in range(26):
    yrs.append(df.take([i]).values[0][0])

fig = go.Figure(
    data=[
        go.Bar(name="Individual", x=years, y=yrs[0:9]),
        go.Bar(name="Organization", x=years, y=[yrs[9]] + [0] + list(yrs[10:17])),
        go.Bar(name="PAC", x=years, y=yrs[17:26]),
    ]
)

fig.update_layout(
    title="Gross Overall Expenses by Donor Type by Year, 2014-22",
    xaxis_title="Year",
    yaxis_title="US Dollars",
)

fig.show()

Above we see overall expenses for PACs and Organizations, as well as total contributions by individuals, splotted year to year. PACs are reliably the biggest spenders, though in some election years they are nearly matched by individual donations. 'Organizations', as categorized by Arizona (being neither political parties, PACs, or vendors) are always the lowest spendders often with nearly no spending outside of election years. In 2015, the Arizona dataset listed no organization spending at all. This spending too is on the rise starting in 2018 

Note that, since no small part of individual contributions go to PACs, the difference in individual and PAC spending may indicate years in which large institutions were more involved in PAC funding. A detailed analysis of this pattern will have to wait until the detailed transaction data is available. 